In [ ]:
import oat_python as oat

import plotly.graph_objects as go
import numpy as np
import networkx as nx
import sklearn
import sklearn.metrics
from sklearn.neighbors import NearestNeighbors
import itertools

# Homology of a Dowker complex

In this example we will
- generate a point cloud
- generate a collection of subsets (some would call this a *cover*, others would call it a *hypergraph*)
- compute the homology of the associated Dowker complex
- analyze cycle representatives
- plot cycle representatives

## Generate a point cloud

In [ ]:
#   USING THE "TORUS CURVE" FUNCTION

cloud   =   oat.point_cloud.torus_curve( nturns=15, angle_initial=0, npoints=300 )
data    =   [   
                go.Scatter3d(
                    x=cloud[:,0],
                    y=cloud[:,1],
                    z=cloud[:,2], 
                    name="Cloud", 
                    mode="markers", 
                    marker=dict(size=5, color=-cloud[:,2], colorscale="Peach"), 
                    showlegend=True,
                    text=[f"Vertex {x}" for x in range(cloud.shape[0])] 
                ) 
            ]

fig = go.Figure( data )
fig.update_layout( 
    title=dict(text="A torus curve"),
    scene = dict(
        aspectratio=go.layout.scene.Aspectratio(x=1, y=1, z=1), # this controls aspect ratio and zoom
        xaxis = dict(range=[-1.5, 1.5],),
        yaxis = dict(range=[-1.5, 1.5],),
        zaxis = dict(range=[-1.5, 1.5],),                
    ),
    width=800, 
    height=800,
    template="plotly_dark",
)    

fig.show()

# Choose an epsilon net

In [ ]:
radius_net          =   0.2; # we'll make epsilon net with this value of epsilon
radius_neighbor     =   0.3; # hyperedges will be neighborhoods of vertices, with this radius

#   COMPUTE AN EPSILON NET
#   ----------------------

# the algorithm uses farthest-point sampling
net, curve          =   oat.dissimilarity.farthest_point_with_cloud( cloud, radius_net )

#   PLOT THE NET
#   ------------

data            =   []
data.append(  
    go.Scatter3d(
        x=cloud[:,0],
        y=cloud[:,1],
        z=cloud[:,2], 
        mode="markers", 
        marker=dict(size=4, symbol="circle", color=-cloud[:,2], colorscale="Peach"),  # Viridis, Pinkyl
        name="Cloud", 
        legendrank=0, 
        opacity=1, 
        text=[f"Vertex {x}" for x in range(cloud.shape[0])] 
    )  
)
data.append(  
    go.Scatter3d(
        x=cloud[net,0],
        y=cloud[net,1],
        z=cloud[net,2], 
        mode="markers", 
        # marker=dict(size=5, symbol="circle",color=cloud[net,2], line=dict(width=4), colorscale="Aggrnyl"),
        marker=dict(size=10, symbol="circle-open",color=-cloud[net,2], line=dict(width=4), colorscale="Peach"), # magenta
        name="Landmarks", 
        legendrank=0, 
        opacity=0.4, 
        text=[f"Vertex {x}" for x in net ] 
    )  
)

fig = go.Figure(data)
fig.update_layout(
    title = f"Landmarks are solid",
    scene = dict(
        aspectratio=go.layout.scene.Aspectratio(x=2.3, y=2.3, z=2.3), # this controls aspect ratio and zoom
        xaxis = dict(range=[-1.75, 1.75],),
        yaxis = dict(range=[-1.75, 1.75],),
        zaxis = dict(range=[-1.75, 1.75],),                
    ),
    width=1000, 
    height=900,
    template="plotly_dark",
)   

fig.show()

## Plot some hyperedges

In [ ]:
#   PLOT THE COVER
#   --------------

data            =   []
data.append(  
    go.Scatter3d(
        x=cloud[:,0],
        y=cloud[:,1],
        z=cloud[:,2], 
        mode="markers", 
        marker=dict(size=5, symbol="circle", color=-cloud[:,2], colorscale="Peach"), 
        name="Cloud", 
        legendrank=0, 
        opacity=0.8, 
        text=[f"Vertex {x}" for x in range(cloud.shape[0])] 
    )  
)
data.append(  
    go.Scatter3d(
        x=cloud[net,0],
        y=cloud[net,1],
        z=cloud[net,2], 
        mode="markers", 
        # marker=dict(size=5, symbol="circle",color=cloud[net,2], line=dict(width=4), colorscale="Aggrnyl"),
        marker=dict(size=10, symbol="circle-open",color=-cloud[net,2], line=dict(width=4), colorscale="Peach"), # magenta
        name="Landmarks", 
        legendrank=0, 
        opacity=0.4, 
        text=[f"Vertex {x}" for x in net ] 
    )  
)

#   hyperedges
for counter, n in enumerate(net):
    trace, x, y , z     =   oat.plot.surface_sphere( 
                                cloud[n][0], 
                                cloud[n][1], 
                                cloud[n][2], 
                                radius=radius_neighbor, 
                                resolution=20 
                            )
    trace.update(
        opacity=1.0, 
        showscale=False, 
        showlegend=True, 
        name=f"Hyperedge {counter}", 
        surfacecolor=z, 
        cmin=-0.6, 
        cmax=0.6 
    )
    if counter % 12 != 0: trace.update(visible="legendonly")
    data.append(trace)

#   figure
fig = go.Figure(data)
fig.update_layout(
    title = f"Several hyperedges",
    scene = dict(
        aspectratio=go.layout.scene.Aspectratio(x=2, y=2, z=2),
        xaxis = dict(range=[-1.75, 1.75],),
        yaxis = dict(range=[-1.75, 1.75],),
        zaxis = dict(range=[-1.75, 1.75],),                
    ),
    width=1100, 
    height=900,
    template="plotly_dark",
)   
fig.show()

## Compute homology

We'll compute the homology of
- the dual hypergraph; that is, the hypergraph where vertices are balls, and for each vertex `v` we have a hyperedge that contains every ball to which `v` belongs
- equivalently, the witness complex where every point is a witness and net points are landmarks

The homology solver only accepts hypergraphs represented by a list of lists of integers, currently.  If your hypergraph has a different format (e.g., if vertices are strings), then you can use some built-in tools to help translate back and forth between this format and list-of-list format; see the [rbs_reduced](rbs_reduced.ipynb) notebook for examples.

In [ ]:
#   COMPUTE THE CORRESPONDING COVER
#   -------------------------------

# data structure holding the whole cloud
net_points          =   NearestNeighbors(n_neighbors=1, algorithm='ball_tree').fit( cloud[net] ) 
# witness complex where all points are witnesses, and net points are landmarks
cover               =   [ sorted(list(x)) for x in net_points.radius_neighbors( cloud, radius=radius_neighbor, return_distance=False, ) ]

#   FACTOR THE BOUNDARY MATRIX
#   --------------------------

factored    =   oat.rust.FactoredBoundaryMatrixDowker( 
                    dowker_simplices            =   cover, 
                    max_homology_dimension      =   2
                )

#   BETTI STATISTICS
#   ----------------

print("\n\nBetti statistics\n")
display( factored.betti() )

#   HOMOLOGY
#   --------

print("\nHomology\n")
homology    =   factored.homology()
display( homology )

#   CYCLE REPRESENTATIVE
#   --------------------

print("\nCycle representative\n")
display( homology["cycle representative"][10] )


## Plot a cycle

In [ ]:
#   DATA FOR THE POINT CLOUD

data            =   []
data.append(  
    go.Scatter3d(
        x=cloud[:,0],
        y=cloud[:,1],
        z=cloud[:,2], 
        mode="markers", 
        marker=dict(size=5, symbol="circle", color=-cloud[:,2], colorscale="Peach"), 
        name="Cloud", 
        legendrank=0, 
        opacity=0.8, 
        text=[f"Vertex {x}" for x in range(cloud.shape[0])] 
    )  
)
data.append(  
    go.Scatter3d(
        x=cloud[net,0],
        y=cloud[net,1],
        z=cloud[net,2], 
        mode="markers", 
        # marker=dict(size=5, symbol="circle",color=cloud[net,2], line=dict(width=4), colorscale="Aggrnyl"),
        marker=dict(size=10, symbol="circle-open",color=-cloud[net,2], line=dict(width=4), colorscale="Peach"), # magenta
        name="Landmarks", 
        legendrank=0, 
        opacity=0.4, 
        text=[f"Vertex {x}" for x in net ] 
    )  
)

#   DATA FOR THE CYCLE

cycle_num           =   2
cycle               =   homology["cycle representative"][cycle_num]
coo                 =   cloud[net] # coo stands for "coordinate oracle"
incident_vertices   =   set()

for counter, linear_term in cycle.iterrows():
    simplex         =   linear_term["simplex"]
    coefficient     =   linear_term["coefficient"]
    incident_vertices.update( simplex )

    trace           =   oat.plot.edge__trace3d( edge=simplex, coo=coo )
    trace.update( name=f"Cycle", text=f"simplex {simplex}<br>linear coefficent {coefficient}", showlegend=True, legendgroup=0, legendrank=1, marker=dict(color="white"), line=dict(width=10))
    if counter != 0: trace.update(showlegend=False)
    data.append(trace) # append to the data group 


incident_vertices   =   [net[x] for x in incident_vertices]
data.append(  go.Scatter3d(x=cloud[incident_vertices,0],y=cloud[incident_vertices,1],z=cloud[incident_vertices,2], mode="markers", marker=dict(color="white"), name="Incident landmark", legendrank=0, text=[f"Vertex {x}" for x in incident_vertices])  )

#   PLOT

fig = go.Figure( data )
fig.update_layout(
    title = f"Toggle the hyperedges!",
    scene = dict(
        aspectratio=go.layout.scene.Aspectratio(x=2.5, y=2.5, z=2.5),
        xaxis = dict(range=[-1.75, 1.75],),
        yaxis = dict(range=[-1.75, 1.75],),
        zaxis = dict(range=[-1.75, 1.75],),                
    ),
    width=800, 
    height=800,
    template="plotly_dark",
)   
fig.update_layout(title="Coefficients appear in hovertext over edges")
fig.update_layout(height=900, width=1100, template="plotly_dark") 
fig.show()

In [ ]:
#   DATA FOR THE POINT CLOUD

data            =   []
data.append(  
    go.Scatter3d(
        x=cloud[:,0],
        y=cloud[:,1],
        z=cloud[:,2], 
        mode="markers", 
        marker=dict(size=5, symbol="circle", color=-cloud[:,2], colorscale="Peach"), 
        name="Cloud", 
        legendrank=0, 
        opacity=0.8, 
        text=[f"Vertex {x}" for x in range(cloud.shape[0])] 
    )  
)
data.append(  
    go.Scatter3d(
        x=cloud[net,0],
        y=cloud[net,1],
        z=cloud[net,2], 
        mode="markers", 
        # marker=dict(size=5, symbol="circle",color=cloud[net,2], line=dict(width=4), colorscale="Aggrnyl"),
        marker=dict(size=10, symbol="circle-open",color=-cloud[net,2], line=dict(width=4), colorscale="Peach"), # magenta
        name="Landmarks", 
        legendrank=0, 
        opacity=0.4, 
        text=[f"Vertex {x}" for x in net ] 
    )  
)

#   DATA FOR THE CYCLE

cycle_num           =   2
cycle               =   homology["cycle representative"][cycle_num]
coo                 =   cloud[net] # coo stands for "coordinate oracle"
incident_vertices   =   set()

for counter, linear_term in cycle.iterrows():
    simplex         =   linear_term["simplex"]
    coefficient     =   linear_term["coefficient"]
    for vertex in simplex: incident_vertices.add(net[vertex])

    trace           =   oat.plot.edge__trace3d( edge=simplex, coo=coo )
    trace.update( name=f"Simplex {simplex}", text=f"simplex {simplex}<br>linear coefficent {coefficient}", showlegend=True, legendrank=1, marker=dict(color="white"), line=dict(width=10))
    data.append(trace) # append to the data group 

incident_vertices   =   list(incident_vertices)
data.append(  go.Scatter3d(x=cloud[incident_vertices,0],y=cloud[incident_vertices,1],z=cloud[incident_vertices,2], mode="markers+text", marker=dict(color="white"), name="Incident landmark", legendrank=0, text=[f"{x}" for x in incident_vertices])  )

#   PLOT

fig = go.Figure( data )
fig.update_layout(
    title = f"Toggle the hyperedges!",
    scene = dict(
        aspectratio=go.layout.scene.Aspectratio(x=2.5, y=2.5, z=2.5),
        xaxis = dict(range=[-1.75, 1.75],),
        yaxis = dict(range=[-1.75, 1.75],),
        zaxis = dict(range=[-1.75, 1.75],),                
    ),
    width=800, 
    height=800,
    template="plotly_dark",
)   
fig.update_layout(title="Coefficients appear in hovertext over edges")
fig.update_layout(height=900, width=1100, template="plotly_dark") 
fig.show()